In [7]:
from jinja2 import Environment, FileSystemLoader
import json

In [8]:
def page_link(a):
    if ' ' in a:
        return a.replace(" ", "_")
    else: 
        return a

In [9]:
f = open('jsons/general.json')
general = json.load(f)

e = open('jsons/research.json')
research = json.load(e)

In [10]:
environment = Environment(loader=FileSystemLoader("templates/"))
environment.globals['page_link'] = page_link
template_no_twitter = environment.get_template("research_page_no_twitter.html.j2")
template_twitter = environment.get_template("research_page_twitter.html.j2")

In [11]:
for project in research:
    filename = f"{ page_link(project['name']) }.html"
    if project["twitter"] == "None":
        content = template_no_twitter.render(
            general=general,
            research=project
        )
        with open(filename, mode="w", encoding="utf-8") as message:
            message.write(content)
    else:
        content = template_twitter.render(
            general=general,
            research=project
        )
        with open(filename, mode="w", encoding="utf-8") as message:
            message.write(content)



In [12]:
f.close()
e.close()
